This notebook calculates similarity and error between protein embeddings and use GO semantic similarity as gold standart.

In [1]:
import pandas as pd
import numpy as np
import gzip
import itertools
import multiprocessing
import csv
import pickle
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm, tqdm_notebook
from multiprocessing import Manager, Pool
from scipy.spatial.distance import cdist
from numpy.linalg import norm
from scipy.stats import spearmanr, pearsonr
import random

## Load protein vectors of ProtVec

In [2]:
ksepVectorRaw = pd.read_csv('/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/Ksep/ksep_dim400.tsv', sep='\t')

In [3]:
ksepVectorRaw

,Protein_Id,Protein_Name,k_separated_bigrams_pssm0,k_separated_bigrams_pssm1,k_separated_bigrams_pssm2,k_separated_bigrams_pssm3,k_separated_bigrams_pssm4,k_separated_bigrams_pssm5,k_separated_bigrams_pssm6,k_separated_bigrams_pssm7,...,k_separated_bigrams_pssm390,k_separated_bigrams_pssm391,k_separated_bigrams_pssm392,k_separated_bigrams_pssm393,k_separated_bigrams_pssm394,k_separated_bigrams_pssm395,k_separated_bigrams_pssm396,k_separated_bigrams_pssm397,k_separated_bigrams_pssm398,k_separated_bigrams_pssm399
0,A0A584,TVBK2_HUMAN,0.5546,0.2142,0.1886,0.3247,0.0797,0.3269,0.3719,0.5601,...,0.6413,0.2945,0.1188,0.2562,0.1925,0.7286,0.8043,0.0753,0.5873,0.4755
1,Q9BXU3,TX13A_HUMAN,3.4557,2.0008,0.7941,1.1575,1.2479,2.3205,2.4250,2.0544,...,1.7612,1.0318,0.5302,0.4156,1.8176,1.7229,0.9445,0.2643,0.2894,1.6563
2,Q15031,SYLM_HUMAN,3.8994,3.4399,2.4516,3.6755,1.0617,2.1469,4.4666,3.1461,...,4.8655,3.6429,1.5017,2.3377,2.8714,3.4553,3.0403,1.3372,1.9682,4.2193
3,Q6PKC3,TXD11_HUMAN,5.8394,3.3412,3.4155,3.7022,0.9748,2.8848,5.6342,3.3171,...,5.9368,4.6732,1.3440,3.2155,2.4538,4.5117,3.3460,0.6302,1.9362,4.7264
4,P42681,TXK_HUMAN,2.0763,2.5351,1.1325,1.5725,0.6736,1.2881,2.0263,1.4697,...,3.0257,2.8324,1.0444,1.1766,1.3982,1.9204,1.5838,1.2209,1.4211,2.2667
5,P40222,TXLNA_HUMAN,3.7529,2.0136,1.9132,2.0341,0.2807,3.5697,6.2421,1.2299,...,1.6613,2.1091,0.3405,0.2619,0.6020,1.3871,0.9950,0.0519,0.1965,0.6723
6,Q8N3L3,TXLNB_HUMAN,4.8453,2.3846,2.2386,2.6784,0.4190,4.7121,7.1746,2.0859,...,2.1459,2.5116,0.5734,0.2879,1.6658,2.0969,1.4097,0.0615,0.2071,0.9678
7,Q86VQ3,TXND2_HUMAN,2.2163,1.2403,1.2934,2.0969,0.2322,1.6010,3.7234,1.6430,...,1.1219,2.3157,0.3923,1.0087,4.5186,2.2724,1.8733,0.1045,0.3905,1.4972
8,Q8NBS9,TXND5_HUMAN,2.6161,1.4423,1.3453,2.0913,0.3304,1.1916,2.6932,1.5698,...,2.2677,2.4352,0.6275,1.4439,1.1129,1.7130,1.3932,0.3893,1.0225,2.1998
9,O14530,TXND9_HUMAN,1.3186,0.9188,0.6860,1.2709,0.2278,0.8875,1.8014,0.7603,...,1.2684,1.0547,0.2978,0.4950,0.4481,0.7678,0.5980,0.0800,0.3530,1.4698


In [4]:
# UNIPROT data for mapping between UNIPROT accession numbers and UNIPROT entry names
uniprot_metadata_directory = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/Uniprot/"
uniprot_metadata_file_path = uniprot_metadata_directory + "uniprot_human_all.tab"
uniprot_vars = ['Entry','Entry name','Status','Protein names','Gene names','Organism','Length','Annotation' ]
uniprot_df = pd.read_csv(uniprot_metadata_file_path, sep='\t', header=None, names=uniprot_vars)

In [5]:
ksepVectorDF = pd.DataFrame(columns=['Entry', 'Vector'])

for index,row in tqdm_notebook(ksepVectorRaw.iterrows()):
    entry = row['Protein_Id']
    vector = list(ksepVectorRaw.iloc[index][2:403])
    ksepVectorDF.loc[index] = [entry,vector]


In [6]:
ksepVectorDF[0:10]

,Entry,Vector
0,A0A584,"[0.5546, 0.2142, 0.1886, 0.3247, 0.0797, 0.326..."
1,Q9BXU3,"[3.4557, 2.0008, 0.7941, 1.1575, 1.2479, 2.320..."
2,Q15031,"[3.8994, 3.4399, 2.4516, 3.6755, 1.0617, 2.146..."
3,Q6PKC3,"[5.8394, 3.3412, 3.4155, 3.7022, 0.9748, 2.884..."
4,P42681,"[2.0763, 2.5351, 1.1325, 1.5725, 0.6736, 1.288..."
5,P40222,"[3.7529, 2.0136, 1.9132, 2.0341, 0.2807, 3.569..."
6,Q8N3L3,"[4.8453, 2.3846, 2.2386, 2.6784, 0.419, 4.7121..."
7,Q86VQ3,"[2.2163, 1.2403, 1.2934, 2.0969, 0.2322, 1.601..."
8,Q8NBS9,"[2.6161, 1.4423, 1.3453, 2.0913, 0.3304, 1.191..."
9,O14530,"[1.3186, 0.9188, 0.6859999999999999, 1.2709, 0..."


In [7]:
ksepVectorDF.to_pickle("/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/embedding_dataframes/ksep_dataframe.pkl")

In [6]:
protein_names = ksepVectorDF['Entry'].tolist()

In [7]:
# define similarity_list and proteinList as global variables
proteinList = []
manager = Manager()
similarity_list = manager.list()
proteinListNew = manager.list()

def parallelSimilarity(paramList):
    i = paramList[0]
    j = paramList[1] 
    aspect = paramList[2]
    if j>i:
        protein1 = proteinListNew[i]
        protein2 = proteinListNew[j]
        if protein1 in protein_names and protein2 in protein_names:
            prot1vec = np.asarray(ksepVectorDF.query("Entry == @protein1")['Vector'].tolist())
            prot2vec = np.asarray(ksepVectorDF.query("Entry == @protein2")['Vector'].tolist())
            #cosine will return in shape of input vectors first dimension
            #print(str(prot1name) + str(prot1vec))
            #print(str(prot2name) + str(prot2vec))
            cos = cosine(prot1vec.reshape(1,-1),prot2vec.reshape(1,-1)).item()
            manhattanDist = cdist(prot1vec.reshape(1,-1), prot2vec.reshape(1,-1), 'cityblock')
            manhattanDistNorm = manhattanDist/(norm(prot1vec,1) + norm(prot2vec,1))
            euclideanDist = cdist(prot1vec.reshape(1,-1), prot2vec.reshape(1,-1), 'euclidean')
            euclideanDistNorm = euclideanDist/(norm(prot1vec,2) + norm(prot2vec,2)) 
            real = paramList[3]
            #real = human_protienSimilarityMatrix.loc[protein1,protein2]
            # To ensure real and calculated values appended to same postion they saved similtanously and then decoupled
            similarity_list.append((real, cos,1-manhattanDistNorm.item(),1-euclideanDistNorm.item()))

    return similarity_list


## Calculate similarity values with parallel processing

In [8]:
def calculateMSEforOntology(aspect,matrix_type):
    
    #Clear lists before each aspect
    similarity_list[:] = []
    proteinListNew[:] = []

    similarityMatrixNameDict = {}
    similarityMatrixNameDict["All"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix.csv" 
    similarityMatrixNameDict["500"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv"
    similarityMatrixNameDict["Sparse"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv" 
    similarityMatrixNameDict["200"] = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix_for_highest_annotated_200_proteins.csv"
            
    similarityMatrixFileName = similarityMatrixNameDict[matrix_type]
        
    human_proteinSimilarityMatrix = pd.read_csv(similarityMatrixFileName)
    human_proteinSimilarityMatrix.set_index(human_proteinSimilarityMatrix.columns, inplace = True)
    #proteinList = human_proteinSimilarityMatrix
    proteinList = human_proteinSimilarityMatrix.columns
    
     #proteinListNew is referanced using Manager
    for prot in proteinList:
        proteinListNew.append(prot)
    if matrix_type == "Sparse":
        #sparsified_similarities = np.load("SparsifiedSimilarites_for_highest_500.npy")
        sparsified_similarity_coordinates = \
        np.load("./auxilary_input/SparsifiedSimilarityCoordinates_"+aspect+"_for_highest_500.npy")
        protParamList = sparsified_similarity_coordinates
    else:     
        i = range(len(proteinList))
        j = range(len(proteinList))
        protParamList = list(itertools.product(i,j))
    protParamListNew = []
    # Prepare parameters for parallel processing these parameters will be 
    # used concurrently by different processes
    for tup in tqdm_notebook(protParamList):
        i = tup[0]
        j = tup[1]
        
        if matrix_type == "Sparse":
            protein1 = proteinListNew[i]
            protein2 = proteinListNew[j]
            real = human_proteinSimilarityMatrix.loc[protein1,protein2]
            tupNew = (tup[0],tup[1],aspect,real)
            protParamListNew.append(tupNew)
        else:
            if j > i:
                protein1 = proteinListNew[i]
                protein2 = proteinListNew[j]
                real = human_proteinSimilarityMatrix.loc[protein1,protein2]
                tupNew = (tup[0],tup[1],aspect,real)
                protParamListNew.append(tupNew)

    total_task_num=len(protParamListNew)
    pool = Pool()
    similarity_listRet = []
    for similarity_listRet in tqdm_notebook(pool.imap_unordered(parallelSimilarity, protParamListNew), total=total_task_num):
        pass
    pool.close()
    pool.join()

    real_distance_list = [value[0] for value in similarity_listRet]
    cosine_distance_list = [value[1] for value in similarity_listRet]
    manhattan_distance_list = [value[2] for value in similarity_listRet]
    euclidian_distance_list = [value[3] for value in similarity_listRet]

    #mseValue = mse(real_distance_list,cosine_distance_list)
    cosineCorr = spearmanr(real_distance_list, cosine_distance_list)
    manhattanCorr = spearmanr(real_distance_list, manhattan_distance_list)
    euclidianCorr = spearmanr(real_distance_list, euclidian_distance_list)   
    
    print("Cosine Correlation for "+aspect+" is " + str(cosineCorr))
    print("Manhattan Correlation for "+aspect+" is " + str(manhattanCorr))
    print("Euclidian Correlation for "+aspect+" is " + str(euclidianCorr))

    return (cosineCorr,manhattanCorr,euclidianCorr)
    

In [ ]:
buffer = "aspect,cosineCorr,cosineCorrPVal,manhattanCorr,manhattanCorrPVal,euclidianCorr,euclidianCorrPVal \n"

for similarity_matrix_type in ["Sparse","200","500","All"]:
    saveFileName = "SimilarityKSEP_"+similarity_matrix_type+".csv"
    f = open(saveFileName,'w')
    f.write(buffer)
    for aspect in ["MF","BP","CC"]:
        corr = calculateMSEforOntology(aspect,similarity_matrix_type) 
        buffer = "" + aspect + ","+ str(corr[0][0])+ ","+ str(corr[0][1])\
        + ","+ str(corr[1][0])+ ","+ str(corr[1][1])+ ","+ str(corr[2][0])+ ","+ str(corr[2][1])+"\n" 
        f = open(saveFileName,'a')
        f.write(buffer) #Give your csv text here.
        ## Python will convert \n to os.linesep
        f.close()
    
# 0.3673674654105104 mse for MF with 0:10
# 0.31965355246378196 mse for BP with 0:10
# 0.29460915219361683 mse for CC with 0:10


Cosine Correlation for MF is SpearmanrResult(correlation=0.15228838647140497, pvalue=0.01683262847017645)
Manhattan Correlation for MF is SpearmanrResult(correlation=0.313503807763848, pvalue=5.189365093137087e-07)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.22896681532294952, pvalue=0.000293311266486756)



Cosine Correlation for BP is SpearmanrResult(correlation=0.28865584496252433, pvalue=3.985785635229992e-06)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.3029230386988472, pvalue=1.2276589635753954e-06)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.3123469039966617, pvalue=5.445185111575878e-07)



Cosine Correlation for CC is SpearmanrResult(correlation=0.2038082020907938, pvalue=0.001369985801290865)
Manhattan Correlation for CC is SpearmanrResult(correlation=0.3293582642528778, pvalue=1.3944105494807714e-07)
Euclidian Correlation for CC is SpearmanrResult(correlation=0.2819688652631666, pvalue=7.714051214210062e-06)



Cosine Correlation for MF is SpearmanrResult(correlation=0.15060923264852613, pvalue=2.6884101283274515e-100)
Manhattan Correlation for MF is SpearmanrResult(correlation=0.12334233160992908, pvalue=1.2184286491297752e-67)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.11713359333018657, pvalue=3.876883219395891e-61)



Cosine Correlation for BP is SpearmanrResult(correlation=0.07889079868238372, pvalue=7.524297378840822e-29)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.08971703015331264, pvalue=7.543392887177284e-37)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.09434875647656171, pvalue=1.3829570993583595e-40)


...


Cosine Correlation for MF is SpearmanrResult(correlation=0.14366047958647976, pvalue=0.0)
Manhattan Correlation for MF is SpearmanrResult(correlation=0.06963852071845925, pvalue=2.8259111125627714e-131)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.06572488597833245, pvalue=4.033508270448133e-117)



Cosine Correlation for BP is SpearmanrResult(correlation=0.056326391894250004, pvalue=3.5954909526322605e-87)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.06283327182416408, pvalue=4.813595199693791e-108)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.07023176520032312, pvalue=1.471893172954835e-134)



Cosine Correlation for CC is SpearmanrResult(correlation=0.0839654418202985, pvalue=1.3508280915756673e-185)
Manhattan Correlation for CC is SpearmanrResult(correlation=0.026205072098513832, pvalue=1.3876932458912944e-19)
Euclidian Correlation for CC is SpearmanrResult(correlation=0.06399565087807534, pvalue=1.7023604962392729e-108)



Cosine Correlation for MF is SpearmanrResult(correlation=0.0938360730669825, pvalue=0.0)
Manhattan Correlation for MF is SpearmanrResult(correlation=-0.009491854841750571, pvalue=5.242559225932143e-94)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.0023170800813545308, pvalue=5.14158111150543e-07)



Cosine Correlation for BP is SpearmanrResult(correlation=0.03655340537916736, pvalue=0.0)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.0024908052411282376, pvalue=3.538850259737585e-27)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.015055657172721214, pvalue=0.0)
